# Modeling on Roseman Stored Data

In this notebook, we will build predictive models based on the cleaned and processed Roseman Stored dataset.

## Objectives
- Select appropriate features for modeling.
- Split the data into training and testing sets.
- Train various machine learning models.
- Evaluate the models' performance.
- Draw insights and recommendations based on the model results.

*Note:* This notebook follows the data cleaning and visualization steps done previously.

### Importing essential libraries for regression modeling

In [1]:
# For data manipulation
import numpy as np
import pandas as pd

# For data preprocessing
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV, cross_val_score
from sklearn.decomposition import PCA

# Regression models
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, f_classif
from xgboost import XGBRegressor

# Model evaluation metrics for regression
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

# Optional: for saving/loading models
import joblib

# Optional: warnings control
import warnings
warnings.filterwarnings('ignore')

### Helper Function

In [2]:
def data_overview(df, name="DataFrame"):
    """
    Show basic structure of the DataFrame:
    - Shape
    - Column names
    - Data types and non-null values
    """
    print(f"\n--- Overview of {name} ---")
    print(f"Shape: {df.shape}")
    print(f"Columns: {list(df.columns)}")
    print("\n--- Info ---")
    print(df.info())


def data_statistics(df, name="DataFrame"):
    """
    Show statistical summary of the DataFrame:
    - Descriptive statistics for all columns
    - Number of unique values per column
    """
    print(f"\n--- Descriptive Statistics of {name} ---")
    print(df.describe(include='all').transpose())
    
    print(f"\n--- Unique Values per Column in {name} ---")
    print(df.nunique().sort_values())


def missing_values_report(df, name="DataFrame"):
    """
    Display a formatted text report of missing values in the DataFrame.
    The output looks like a table, but it's printed as plain text.
    """
    print(f"\n--- Missing Values in {name} ---")
    missing_count = df.isnull().sum()
    missing_percent = (missing_count / len(df)) * 100

    # Keep only columns with missing values
    mask = missing_count > 0
    if mask.sum() == 0:
        print("No missing values detected.")
        return

    # Print table header
    print(f"{'':<18}{'Missing Count':>15}  {'Missing %':>10}")

    # Print each row aligned
    for col in df.columns[mask]:
        count = missing_count[col]
        percent = missing_percent[col]
        print(f"{col:<18}{count:>15}  {percent:>10.6f}")


def show_value_counts(df, columns):
    """
    Display value counts for a list of columns in a DataFrame.

    Parameters:
    df (pd.DataFrame): The input DataFrame
    columns (list): A list of column names for which to show value counts
    """
    for col in columns:
        print(f"--- Value Counts for column: '{col}' ---")
        print(df[col].value_counts())
        print("-" * 50)


def train_evaluate_model(model, x_train, y_train, x_test, y_test):
    """
    Train the given model on training data, predict on both training and test data,
    then calculate and print MAE, RMSE, and R2 metrics for both datasets.
    
    Parameters:
    - model: sklearn-compatible regression model instance
    - x_train: training features
    - y_train: training targets
    - x_test: test features
    - y_test: test targets
    """
    model.fit(x_train, y_train)
    y_train_pred = model.predict(x_train)
    y_test_pred = model.predict(x_test)

    mae_train = mean_absolute_error(y_train, y_train_pred)
    rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
    r2_train = r2_score(y_train, y_train_pred)

    mae_test = mean_absolute_error(y_test, y_test_pred)
    rmse_test = np.sqrt(mean_squared_error(y_test, y_test_pred))
    r2_test = r2_score(y_test, y_test_pred)

    print(f"{model.__class__.__name__} Performance Metrics\n")
    print("Training Set:")
    print(f"  Mean Absolute Error (MAE): {mae_train:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse_train:.4f}")
    print(f"  R-squared Score (R2): {r2_train:.4f}\n")
    print("Test Set:")
    print(f"  Mean Absolute Error (MAE): {mae_test:.4f}")
    print(f"  Root Mean Squared Error (RMSE): {rmse_test:.4f}")
    print(f"  R-squared Score (R2): {r2_test:.4f}")

### Load Cleaned Datasets for Further Processing

In [3]:
# Load cleaned training data
clean_data = pd.read_csv(r"clean_data.csv")

# Load raw test data
test_data = pd.read_csv(r"test_data.csv")

# Merge both datasets to apply unified preprocessing
df = pd.concat([clean_data, test_data], axis=0).reset_index(drop=True)

### Data Preprocessing: Dropping Irrelevant Columns and Encoding Categorical Variables

In [4]:
# Use data overview function to display basic info about the cleaned data
data_overview(df, name="Full Data")


--- Overview of Full Data ---
Shape: (1058297, 23)
Columns: ['Store', 'DayOfWeek', 'Date', 'Sales', 'Customers', 'Open', 'Promo', 'StateHoliday', 'SchoolHoliday', 'StoreType', 'Assortment', 'compdistance', 'compmonth', 'compyear', 'Promo2', 'Promo2SinceWeek', 'Promo2SinceYear', 'PromoInterval', 'is_test', 'Year', 'Month', 'Day', 'Day_name']

--- Info ---
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1058297 entries, 0 to 1058296
Data columns (total 23 columns):
 #   Column           Non-Null Count    Dtype  
---  ------           --------------    -----  
 0   Store            1058297 non-null  int64  
 1   DayOfWeek        1058297 non-null  int64  
 2   Date             1058297 non-null  object 
 3   Sales            1017209 non-null  float64
 4   Customers        1017209 non-null  float64
 5   Open             1058297 non-null  float64
 6   Promo            1058297 non-null  int64  
 7   StateHoliday     1058297 non-null  object 
 8   SchoolHoliday    1058297 non-null  int64  
 

In [5]:
# Use data statistics function to show descriptive stats and unique values
data_statistics(df, name="Full Data")


--- Descriptive Statistics of Full Data ---
                     count unique         top     freq         mean  \
Store            1058297.0    NaN         NaN      NaN   558.331493   
DayOfWeek        1058297.0    NaN         NaN      NaN     3.997596   
Date               1058297    990  2015-07-31     1115          NaN   
Sales            1017209.0    NaN         NaN      NaN  5773.818972   
Customers        1017209.0    NaN         NaN      NaN    -1.364282   
Open             1058297.0    NaN         NaN      NaN     0.831048   
Promo            1058297.0    NaN         NaN      NaN      0.38207   
StateHoliday       1058297      5           0   625531          NaN   
SchoolHoliday    1058297.0    NaN         NaN      NaN     0.188929   
StoreType          1058297      4           a   573755          NaN   
Assortment         1058297      3           a   557749          NaN   
compdistance     1058297.0    NaN         NaN      NaN    -0.048868   
compmonth        1058297.0    Na

### Preprocessing: Dropping & Encoding

In [6]:
# Generate a missing values report for the cleaned dataset
missing_values_report(df, name="Full Data")


--- Missing Values in Full Data ---
                    Missing Count   Missing %
Sales                       41088    3.882464
Customers                   41088    3.882464


In [7]:
# Drop 'Date', 'DayOfWeek' and is_test columns because their information is already captured by 'Year', 'Month', 'Day', and other features.
# Keeping these columns would be redundant.
columns_to_drop = ['Date', 'DayOfWeek', 'is_test']

df.drop(columns=columns_to_drop, axis=1, inplace=True)

In [8]:
# Extract columns with categorical data type
categorical_columns = df.select_dtypes(include=['object']).columns.tolist()

# Generate value counts report for categorical columns
show_value_counts(df, categorical_columns)

--- Value Counts for column: 'StateHoliday' ---
StateHoliday
0    625531
0    401536
a     20440
b      6690
c      4100
Name: count, dtype: int64
--------------------------------------------------
--- Value Counts for column: 'StoreType' ---
StoreType
a    573755
d    327024
c    141112
b     16406
Name: count, dtype: int64
--------------------------------------------------
--- Value Counts for column: 'Assortment' ---
Assortment
a    557749
c    491822
b      8726
Name: count, dtype: int64
--------------------------------------------------
--- Value Counts for column: 'PromoInterval' ---
PromoInterval
noPromo             525263
Jan,Apr,Jul,Oct     306898
Feb,May,Aug,Nov     124308
Mar,Jun,Sept,Dec    101828
Name: count, dtype: int64
--------------------------------------------------
--- Value Counts for column: 'Day_name' ---
Day_name
Thursday     151837
Wednesday    151657
Tuesday      151656
Friday       150981
Monday       150722
Sunday       150722
Saturday     150722
Name: count

In [ ]:
# Map StateHoliday categorical values to numerical codes
stateholiday_map ={'0': 0, 'a': 1, 'b': 2, 'c': 3}
df['StateHoliday'] = df['StateHoliday'].map(stateholiday_map)

# Map Assortment categorical values to numerical codes
assortment_map = {'a': 0, 'b': 1, 'c': 2}
df['Assortment'] = df['Assortment'].map(assortment_map)

In [10]:
# Apply one-hot encoding to 'Day_name', 'StoreType' and 'PromoInterval' columns
# - dtype='int64' ensures resulting dummy variables are integers, which saves memory and is often preferred for ML models
# - drop_first=True avoids dummy variable trap (multicollinearity) by dropping the first category from each encoded column
df = pd.get_dummies(df, columns=['StoreType', 'PromoInterval', 'Day_name'], dtype='int64', drop_first=True)

In [11]:
# Calculate correlation between each feature and 'Sales'
sales_correlations = df.corr(numeric_only=True)['Sales'].sort_values(ascending=False)

# Display correlations
print("Correlation between each feature and 'Sales':\n")
print(sales_correlations)

Correlation between each feature and 'Sales':

Sales                             1.000000
Customers                         0.767544
Open                              0.678472
Promo                             0.452345
Day_name_Monday                   0.215309
StoreType_b                       0.139940
Day_name_Tuesday                  0.130764
PromoInterval_noPromo             0.091040
SchoolHoliday                     0.085124
Day_name_Wednesday                0.083047
Assortment                        0.074941
Day_name_Thursday                 0.050344
Month                             0.048768
Year                              0.023519
compyear                          0.010115
Day_name_Saturday                 0.007801
Store                             0.005126
StoreType_c                      -0.005140
Day                              -0.011612
StoreType_d                      -0.022854
compmonth                        -0.023638
PromoInterval_Jan,Apr,Jul,Oct    -0.034946
Promo2S

In [12]:
# Drop 'Customers' column because it is missing in test data 
# and is strongly correlated with the target variable 'Sales',
# which makes it redundant and potentially misleading for model training.
df.drop(columns=['Customers'], inplace=True)

In [13]:
# Split the dataset into train and test sets based on missing values in 'Sales'
# Rows with non-missing 'Sales' will be used for training
# Rows with missing 'Sales' will be used for testing (unlabeled data)
train_data = df[df['Sales'].notnull()].copy()
test_data = df[df['Sales'].isnull()].copy()

# Optional: reset index for both sets
train_data.reset_index(drop=True, inplace=True)
test_data.reset_index(drop=True, inplace=True)

In [14]:
# Generate a missing values report for the train and test dataset
missing_values_report(train_data, name="Train Dataset")
missing_values_report(test_data, name="Test Dataset")


--- Missing Values in Train Dataset ---
                    Missing Count   Missing %
StateHoliday               393216   38.656363

--- Missing Values in Test Dataset ---
                    Missing Count   Missing %
Sales                       41088  100.000000
StateHoliday                 8320   20.249221


### Data Splitting, Scaling & PCA

In [15]:
# Split the data into features (X) and target (y)
x = train_data.drop('Sales', axis=1)  # Drop the target column from the features
y = train_data['Sales']  # Extract the target column

# Split the data into training and testing sets (85% train, 15% test)
x_train, x_test, y_train, y_test = train_test_split(
    x, y, test_size=0.15, random_state=42
)

In [16]:
# Apply StandardScaler to scale features for both training and testing sets
scaler = StandardScaler()

# Fit the scaler on the training features and transform them
x_train_scaled = scaler.fit_transform(x_train)

# Transform the testing features using the same scaler fitted on training data
x_test_scaled = scaler.transform(x_test)

In [17]:

print(np.isnan(x_train_scaled).sum())  # nunmber of mussing values



334093


In [18]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean')  
x_train_scaled = imputer.fit_transform(x_train_scaled)
x_test_scaled = imputer.transform(x_test_scaled)


In [19]:
# Apply PCA for dimensionality reduction after StandardScaler
pca = PCA(n_components=0.90)  # Keep 90% of variance

# Fit PCA on scaled training data and transform it
x_train_pca = pca.fit_transform(x_train_scaled)

# Transform the scaled testing data using the same PCA model
x_test_pca = pca.transform(x_test_scaled)

In [20]:
# Check the shape of the PCA-transformed training and testing data
print("Shape of X_train after PCA:", x_train_pca.shape)
print("Shape of X_test after PCA:", x_test_pca.shape)

Shape of X_train after PCA: (864627, 18)
Shape of X_test after PCA: (152582, 18)


### Model Training & Evaluation

In [21]:
import numpy as np
print("NaNs in y_train:", np.isnan(y_train).sum())
print("NaNs in y_test: ", np.isnan(y_test).sum())


NaNs in y_train: 0
NaNs in y_test:  0


In [22]:
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression

imputer = SimpleImputer(strategy='mean')
x_train_i = imputer.fit_transform(x_train)
x_test_i  = imputer.transform(x_test)

train_evaluate_model(LinearRegression(), x_train_i, y_train, x_test_i, y_test)


LinearRegression Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 1747.0587
  Root Mean Squared Error (RMSE): 2494.4812
  R-squared Score (R2): 0.5804

Test Set:
  Mean Absolute Error (MAE): 1739.1423
  Root Mean Squared Error (RMSE): 2483.2273
  R-squared Score (R2): 0.5825


In [23]:
df_train = pd.DataFrame(x_train, columns=[f'feat_{i}' for i in range(x_train.shape[1])])
df_test  = pd.DataFrame(x_test,  columns=[f'feat_{i}' for i in range(x_test.shape[1])])

print("Missing in TRAIN per column:")
print(df_train.isna().sum())

print("\nMissing in TEST per column:")
print(df_test.isna().sum())


Missing in TRAIN per column:
feat_0     864627
feat_1     864627
feat_2     864627
feat_3     864627
feat_4     864627
feat_5     864627
feat_6     864627
feat_7     864627
feat_8     864627
feat_9     864627
feat_10    864627
feat_11    864627
feat_12    864627
feat_13    864627
feat_14    864627
feat_15    864627
feat_16    864627
feat_17    864627
feat_18    864627
feat_19    864627
feat_20    864627
feat_21    864627
feat_22    864627
feat_23    864627
feat_24    864627
feat_25    864627
feat_26    864627
dtype: int64

Missing in TEST per column:
feat_0     152582
feat_1     152582
feat_2     152582
feat_3     152582
feat_4     152582
feat_5     152582
feat_6     152582
feat_7     152582
feat_8     152582
feat_9     152582
feat_10    152582
feat_11    152582
feat_12    152582
feat_13    152582
feat_14    152582
feat_15    152582
feat_16    152582
feat_17    152582
feat_18    152582
feat_19    152582
feat_20    152582
feat_21    152582
feat_22    152582
feat_23    152582
feat_24    

In [24]:
from sklearn.impute import SimpleImputer

imputer = SimpleImputer(strategy='mean') 
x_train = imputer.fit_transform(x_train)
x_test = imputer.transform(x_test)



In [25]:
# Initialize the Linear Regression model
model = LinearRegression()

# Train the model and evaluate it using the predefined function
train_evaluate_model(model, x_train, y_train, x_test, y_test)

LinearRegression Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 1747.0587
  Root Mean Squared Error (RMSE): 2494.4812
  R-squared Score (R2): 0.5804

Test Set:
  Mean Absolute Error (MAE): 1739.1423
  Root Mean Squared Error (RMSE): 2483.2273
  R-squared Score (R2): 0.5825


In [26]:
# Initialize the XGBoost Regressor model
model = XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='rmse')

# Train and evaluate using the reusable function
train_evaluate_model(model, x_train, y_train, x_test, y_test)

XGBRegressor Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 774.4136
  Root Mean Squared Error (RMSE): 1107.2430
  R-squared Score (R2): 0.9173

Test Set:
  Mean Absolute Error (MAE): 780.3175
  Root Mean Squared Error (RMSE): 1120.2096
  R-squared Score (R2): 0.9150


In [27]:
# Define parameter grid for Random Search
param_dist = {
    'n_estimators': [100, 200, 300, 400, 500],
    'max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.6, 0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.6, 0.7, 0.8, 0.9, 1.0],
    'gamma': [0, 0.1, 0.2, 0.3, 0.5],
    'reg_alpha': [0, 0.01, 0.1, 1],
    'reg_lambda': [0.5, 1, 1.5, 2],
}

# Randomized Search CV setup
random_search = RandomizedSearchCV(
    estimator=XGBRegressor(random_state=42, use_label_encoder=False, eval_metric='rmse'),
    param_distributions=param_dist,
    n_iter=50,
    scoring='neg_root_mean_squared_error',
    cv=3,
    verbose=1,
    random_state=42,
    n_jobs=-1
)

# Fit the search on training data
random_search.fit(x_train, y_train)

# Get the best model from search
best_model = random_search.best_estimator_

# Train and evaluate using the provided function
train_evaluate_model(best_model, x_train, y_train, x_test, y_test)

Fitting 3 folds for each of 50 candidates, totalling 150 fits
XGBRegressor Performance Metrics

Training Set:
  Mean Absolute Error (MAE): 358.6147
  Root Mean Squared Error (RMSE): 544.4490
  R-squared Score (R2): 0.9800

Test Set:
  Mean Absolute Error (MAE): 410.6212
  Root Mean Squared Error (RMSE): 651.1451
  R-squared Score (R2): 0.9713


### Save the trained model, PCA transformer, and Standard Scaler for future use

In [28]:
# Save the best trained regression model to disk
joblib.dump(best_model, 'XGB_Model.pkl')

# Save the fitted StandardScaler to disk
joblib.dump(scaler, 'standard_scaler.pkl')

# Save the fitted PCA transformer to disk
joblib.dump(pca, 'pca_transformer.pkl')

print("Best model, scaler, and PCA have been saved successfully.")

Best model, scaler, and PCA have been saved successfully.


### Save final test data with predictions

In [29]:
# Drop the 'Sales' column from the test_data DataFrame if it exists
test_data = test_data.drop(columns=['Sales'], axis=1)

In [30]:
test_data.shape

(41088, 27)

In [31]:
x_train.shape

(864627, 27)

In [32]:
# Make predictions
predictions = model.predict(test_data)

# Add predictions as a new column in test_data
test_data['Sales'] = predictions

# Show updated DataFrame (optional)
print(test_data.head())

   Store  Open  Promo  StateHoliday  SchoolHoliday  Assortment  compdistance  \
0      1   1.0      1           0.0              0           0     -0.267367   
1      3   1.0      1           0.0              0           0      0.794428   
2      7   1.0      1           0.0              0           2      1.027952   
3      8   1.0      1           0.0              0           0      0.516402   
4      9   1.0      1           0.0              0           2     -0.060734   

   compmonth  compyear  Promo2  ...  PromoInterval_Jan,Apr,Jul,Oct  \
0        9.0    2008.0       0  ...                              0   
1       12.0    2006.0       1  ...                              1   
2        4.0    2013.0       0  ...                              0   
3       10.0    2014.0       0  ...                              0   
4        8.0    2000.0       0  ...                              0   

   PromoInterval_Mar,Jun,Sept,Dec  PromoInterval_noPromo  Day_name_Monday  \
0                    

In [33]:
# Export the test data along with the predicted sales to a CSV file for future reporting or analysis
test_data.to_csv('XGB_TestData_With_Predicted_Sales.csv', index=False)